# JEM092 Asset Pricing - Homework 2

### Summer Semester 2021/2022

### Authors: *Petr Čala, Tereza Čechová, Vilém Krejcar*
___

## Introduction

In this homework, we are going to try and take a look at how different factors influence portfolio performance. This we will do in three steps.

First we will run a univariate portfolio sort on these factors - beta, size, and volatility. Splitting the data of 250 stocks into 5 quintiles based on these factors, we will calculate one-month ahead excess returns for each period. First we will weight these equally, and later by firm market capitalization, to capture potential effect of large firms. We will then compare the average returns for all portfolio quintiles, and see, whether these are somehow influenced by other market factors, such as the FF3 factors, momentum, or illiquidity.

Second we will perform an independent, value-weighted, bivariate portfolio sort on a combination of the three main factors. Observing virtually the same metrics, we will try to explore, whether introducing more breakpoints into the portfolio sorting leads to different results or new trends in the data. One-month ahead excess returns will be considered and compared to one-quarter ahead excess returns.

Lastly, the effect of volatility on returns will be examined alone, as well as paired with other aforementioned factors, using Fama-MacBeth regressions.

## Data Description

Although we were able to obtain the desired data sets in the previous homework, we decided to use the provided `Asset_Pricing_HW_2_data.RData` file, which allows for easier and more reliable replicability. One caveat in this approach appeared, however, when an MOH ticker assigned to our group was not provided in said data file. We decided to remedy this by arbitrarily choosing a different stock data from said file instead. We thus obtained a randomly (or semi-randomly, but without introducing bias in the data) selected **data of 250 stocks**, namely the O-H-L-C-V data, the market capitalization data, and the book-to-market ratio data.

For our purposes, several transformation had to be made. We first calculated daily returns from the OHLCV data to be used when calculating monthly betas, volatility and illiquidity. This was done in a straightforward manner using adjusted prices from the OHLCV data set. Then we calculated these **returns at monthly intervals**, which served as a backbone to the whole homework, being the **main dependent variable in most estimations**. We also transformed the market capitalization data into monthly periods, which we later on used in weighting the returns of different portfolios in both univariate and bivariate portfolio sort.

As for the sorting factors, we first wrote a function `getFactor`, which is able to fetch the following into a nice `.xts` dataset - FF3 factors (daily or monthly), Momentum, Illiquidity. However, a large problem with these data was that **the online momentum and illiquidity factors are provided only for a pre-defined portfolio, and not all stocks we have in our data set**. We thus had to add two more functions - `getMonthlyMom` aand `getMonthlyIlliq`, which take the returns data for our stocks and returns an `.xts` object with said two factors for all of these stocks. The online factor data sets are used in the two portfolio sorts' FFCPS regression, while the utility of the latter data set (computed manually) can be found in the Fama-MacBeth regression construction.

As a last data preparation procedure, we calculated the three sort factors - **beta, size, and volatility**. For the first and the third, we used last 12 months of data when calculating returns, as suggested in the setup. The rest of the transformation was quite straightforward.

Also, a *note should be made* on the time range in which we conduct our calculations. We set no fixed range for any of the data sets, which means their lengths differ, at times greatly. We believe, however, that this should bring no historical discrepancy bias into our estimations. The reason for this lies in the fact that when constructing the portfolio sort, or the FMB regressions, **we always subset to the index of available monthly returns**, which does not change across methods. This period falls roughly into the range of 2000-2022 data, which we believe should not cause any major changes in the outcome trends. As we noticed the setup specified sample period only after writing all of the code, **backpropagating this change seemed of little necessity**, with the aforementioned reasoning in mind.

## Empirical methodology

The steps conducting in the analysis part of this homework are three-fold.

#### Univariate sort

**First, we construct a three univariate portfolio sorts**, each time on a different factor. These are - beta, size, and volatility. For each of these, 5 quintiles are calculated at each period of monthly returns, based on which the stocks are sorted in to 5 portfolios. For each of these, average one-month ahead excess return is then calculated, first in an equal manner, and later weighted by the market capitalization of each company. We then take the time series average return for all 5 quintiles and report it along with the corresponding NeweyWest  (NW) t-statistic.

Furthermore, we look at the average returns of the difference portfolio (in our case 5-1), along with its t-statistic, and lastly we also report the $\alpha$ from a regression of the excess returns on five factors - excess market returns, High-Minus-Low, Small-Minus-Big (which are the 3 Fama-French factors), Momentum (Carhart), and Illiquidity (Pastor-Stambaugh), alongh with its NW t-statistic. As an addition, we also report the average quintile values for each of the three sorting factors.

The actual estimation ended up being, although lengthy, decently straightforward. We treat periods with little to no observations adequately (which means we skip them), calculate the time series of monthly returns, calculate the average + NW t-statistic, add the difference portfolio and FFCPS regression alpha. This we run for all three factors, and encompass the actual estimation, along with the output generation, in two generalizable functions - `getUnivariateSort` and `getVariateSortOutput`. The code includes commends indicating what part of the process happends where, which makes it easy to read and navigate in.

#### Bivariate sort

The second of our major contributions lies in constructing a **bivariate, value-weighted, independent sort**, this for two pairs of sorting factors - **first the Size-Beta pair, and second for the Size-Volatility pair**. Three quantiles are defined for this sort, and a single to achieve similar looking output to the univariate sort (`getBivariateSort`). Time series average returns and NW-test statistics are reported for all 9 portfolio breakpoint *tiles*, along with 6 difference portfolio (3-1, pairwise) and 6 FFCPS alpha average returns plus NW t-statistics. **These all are conducted twice**, once looking at the one-month ahead excess returns, and once looking at the one-quarter ahead excess returns.

Based on the inherent property of the independent sorting, each of the 9 portfolios will end up with a different amount of stocks in each period, from which the returns are calculated. This could end up posing problems in case of **insufficient number of stocks in empirically rare factor combinations** (such as low volatility, low size), but **for us this was not such**. Similarly to the univariate sort, where observations were insufficient, no returns were calculated, leaving the resulting average unaffected. As we do get a result for all portfolios in all methods, this concern proved to be insignificant.

As a side note, we would like to include a fun trivia - the `getBivariateSort` function is the lengthiest of functions the coding person assigned to that part has written in all of their coding career, which they do for a living.

#### Fama-MacBeth regressions

As a last contribution of our homework, we run a battery of Fama-MacBeth regressions with the goal of **trying to explain the effect of volatility using other sorting factors**. These include - beta, size, momentum, and illiquidity. We first run the regression of returns only on volatility, then combine volatility with each of the remaining factors one at a time, and lastly run a regression with all factors present at once.

Peculiarly, we saw the setup trying to suggest a regression on the three F-F factors, which made little sense to us, as those are not easily available online for all 250 stocks, which is what we would need. We thus decided to remove these factors completely from the regressions. Similarly, as outline in the *Data description* section of this homework, the data for momentum and illiquidity were unavailable for all 250 stocks online, so we had to manually compute it based on the supplied data sets.

As such, we obtained 6 different regressions, from all of which we report the time series average of the resulting coefficients of all present factors for each regression (along with the intercept), along with the corresponding NW t-statistics.

## Results

#### Univariate sort

The results to look for in the univariate sort can be divided into several simple categories. We are first interested whether sorting by different factor yields to different outcomes. Then we want to see if the FFCPS $\alpha$ can explain the excess return behavior, which is not caused by sensitivity to other factors. Last cause for our attention will be the effect of weighting the returns by market capitalization. Displayed below are the average value for all three factors across quantiles (table 1).

<br>
<center><i><b>Table 1: Quintile average values</b></i></center>

| Factor | 1 | 2 | 3 | 4 | 5 |
|---|---|---|---|---|---|
| Beta | 0.4858 | 0.7563 | 0.9418 | 1.1482 | 1.5955 |
| Size | 1.4119 | 2.1240 | 2.6941 | 3.3453 | 4.5757 |
| Volatility | 4.3400 | 5.5260 | 6.4991 | 7.8382 | 11.2847 |

To answer the first of the questions - how the returns change after controlling for each of the three factors. **The theory predicts** that the firms with highest returns will be those where for the respective factors the following holds - **high beta, small size, high volatility**. To provide quick reasoning for this; a stock with high beta responds strongly to the market, which comes with higher risk, and logically higher potential returns. As for the latter two factors, smaller firms have smaller stock volume, and are more prone to sudden growth or loss, which makes them more volatile, all the while holding higher potential for high returns. In macro economics, this is sometimes called the catch-up effect.

<br>
<center><i><b>Table 2: Univariate sort - equally weighted</b></i></center>

| Factor | Metric | 1 | 2 | 3 | 4 | 5 | 5 - 1 | FFCPS alpha |
|---|---|---|---|---|---|---|---|---|
| Beta | Average Returns | 0.0102 | 0.0125 | 0.0133 | 0.0169 | 0.0174 | 0.0072 | 0.0076 |
| Beta | NeweyWest t-stat | (4.5661) | (3.9316) | (3.5691) | (3.6490) | (2.9726) | (1.5407) | (1.5634) |
| Size | Average Returns | 0.0258 | 0.0144 | 0.0129 | 0.0109 | 0.0099 | -0.0160 | -0.0157 |
| Size | NeweyWest t-stat | (4.8098) | (3.2872) | (2.7436) | (3.1781) | (2.7838) | (-5.1373) | (-5.2873) |
| Volatility | Average Returns | 0.0104 | 0.0126 | 0.0137 | 0.0163 | 0.0205 | 0.0101 | 0.0106 |
| Volatility | NeweyWest t-stat | (4.7774) | (4.2211) | (3.7720) | (3.8077) | (3.5090) | (2.2479) | (2.1967) |

<br>
<center><i><b>Table 3: Univariate sort - value-weighted</b></i></center>

| Factor | Metric | 1 | 2 | 3 | 4 | 5 | 5 - 1 | FFCPS alpha |
|---|---|---|---|---|---|---|---|---|
| Beta | Average Returns | 0.0109 | 0.0112 | 0.0109 | 0.0159 | 0.0215 | 0.0107 | 0.0109 |
| Beta | NeweyWest t-stat | (4.3326) | (3.8501) | (2.9751) | (3.4958) | (3.8753) | (2.0778) | (1.9444) |
| Size | Average Returns | 0.0311 | 0.0199 | 0.0176 | 0.0146 | 0.0126 | -0.0185 | -0.0186 |
| Size | NeweyWest t-stat | (5.1604) | (4.4502) | (3.8404) | (4.3147) | (3.9384) | (-4.6305) | (-5.1836) |
| Volatility | Average Returns | 0.0090 | 0.0111 | 0.0151 | 0.0159 | 0.0249 | 0.0159 | 0.0171 |
| Volatility | NeweyWest t-stat | (3.7261) | (3.2137) | (4.2642) | (3.6027) | (4.4187) | (3.2809) | (3.3304) |

As we can see above in the two tables displaying the results of the univariate portfolio analysis (table 2, table 3), all three of these predictions are clearly visible in our results. Moreover, literally all of these are statistically significant as indicated by the NeweyWest t-statistic. **The underlying effect thus seem to be strong, resolute, and valid on average across time**.

When we look only at the difference portfolio, however, the statistical significance drops at times, most notably in the both weighting cases of the beta sort. This is, however, by no means a strong evidence in the opposite direction, so there should be little to no ambiguity on the behavior of the effect of sorting by each of the three factors.

Glancing at the FFCPS $\alpha$, no signicant differences are to be spotted for any factors in either of the tables. Just as a reminder, the factors on which we are regressing the returns, are market excess returns, High-Minus-Low, Small-Minus-Big (FF3), Momentum (Pastor), and Illiquidity (Pastor-Stambaugh). It thus seems, that **the behavior pattern of one-month ahead excess returns remains very similar, even when controlled for possible influence of these factors**.

Lastly, we would like to compare the outcome of the two portfolio sort groups, one done with equally weighted returns, and one with value-weighted ones. As outlined before, the individual factor effect directions remain the same (such as high size leads to lower returns, etc.) for both weighting schemes. However, **the magnitude of the returns is usually slightly lower for the value-weighted process**, than its counterpart (apart from when sorting by volatility). Nonetheless, this magnitude is of a rather smaller size, meaning that the returns suddenly do not disapper when weighted by stock market capitalization, or anything like that. This, we feel like, is the most important message, rather than decimal-spot nuances in the trends. That is - **the trends are likely not driven by the largest stocks, but are rather present across the data as a whole**. Still, we can see a slight overall drop in returns when attributing larger weight to firms with higher market cap, which is what the theory would predict.

We could go into more details discussing individual quantile behavior, but we feel like we have sufficiently explained the general trends, which is our primary objective for this section.

#### Bivariate sort

Here our objectives will be the following - see how the trends move when looking at either direction (meaning in direction of the one or the other factor), whether other factors can explain this behavior (FFCPS $\alpha$), if sorting by different pairs of factors leads to different results, and lastly, whether calculating with returns further in the future makes any difference.

First, we display the results for one-month ahead bivariate independent value-weighted portfolio sort, first on the size-beta pair (table 4), and second on the size-volatility pair (table 5).

<br>
<center><i><b>Table 4: Bivariate sort - size-beta - one-month ahead excess returns</b></i></center>

| Portfolio | Metrics | Size 1 | Size 2 | Size 3 | Size 3 - 1 | Size 3 - 1 FFCPS alpha |
|---|---|---|---|---|---|---|
| Beta 1 | Average Returns | 0.0188 | 0.0112 | 0.0110 | -0.0078 | -0.0072 |
| Beta 1 | N-W t-stat | (5.8663) | (4.2603) | (4.7210) | (-4.1704) | (-3.551) |
| Beta 2 | Average Returns | 0.0226 | 0.0151 | 0.0120 | -0.0106 | -0.0068 |
| Beta 2 | N-W t-stat | (4.5934) | (3.5177) | (3.1402) | (-4.2714) | (-3.2791) |
| Beta 3 | Average Returns | 0.0360 | 0.0214 | 0.0176 | -0.0184 | -0.0085 |
| Beta 3 | N-W t-stat | (4.9506) | (3.6224) | (3.6945) | (-4.5138) | (-2.304) |
| Beta 3 - 1 | Average Returns | 0.0172 | 0.0102 | 0.0066 | | |
| Beta 3 - 1 | N-W t-stat | (2.9621) | (2.2228) | (1.6235) | | |
| Beta 3 - 1 FFCPS alpha | Average Returns | 0.0114 | 0.0061 | 0.0101 | | |
| Beta 3 - 1 FFCPS alpha | N-W t-stat | (2.2462) | (1.2058) | (2.1869) | | |

Observing first the general direction trends for the size-beta sort - the effect of "smaller firms $\approx$ higher returns" persist across all quantiles of beta, for which the effect is still "higher beta $\approx$ higher returns". As should be expected, the **largest average returns are found in the quantile with firms possessing (on average) the highest beta and the smallest size, while the lowest average returns belong to the opposite quantile (average smallest beta, largest size)**. All of these effects are also statistically highly significant.

As for the difference portfolios, a notable loss of significance occurs at the beta difference portfolio. Indeed, this mimics the univariate sort case, possibly meaning that the average difference in returns is not as large, when changing purely the firms's beta.

An interesting observation can be made about the FFCPS $\alpha$, as the intercept of the regression of returns on the five control factors is notably lower than the difference portfolio returns, and statistically significant at that. That means that **some of the explanatory factors hold an *explanatory* power in the size of the returns**. In other words, the differences between returns of individual quantiles may not be as pronounced, as the simple sorting analysis would suggest. It is difficult to say, which of these factors (if one, or more) is causing this, but we will explore this further in the Fama-MacBeth regreession section later in the homework.

<br>
<center><i><b>Table 5: Bivariate sort - size-volatility - one-month ahead excess returns</b></i></center>

| Portfolio | Metrics | Size 1 | Size 2 | Size 3 | Size 3 - 1 | Size 3 - 1 FFCPS alpha |
|---|---|---|---|---|---|---|
| Volatility 1 | Average Returns | 0.0164 | 0.0112 | 0.0100 | -0.0064 | -0.0052 |
| Volatility 1 | N-W t-stat | (5.7918) | (4.2531) | (3.6592) | (-4.7271) | (-3.045) |
| Volatility 2 | Average Returns | 0.0225 | 0.0142 | 0.0133 | -0.0091 | -0.007 |
| Volatility 2 | N-W t-stat | (4.8468) | (3.3325) | (3.6714) | (-3.6964) | (-3.2542) |
| Volatility 3 | Average Returns | 0.0353 | 0.0223 | 0.0207 | -0.0147 | -0.0052 |
| Volatility 3 | N-W t-stat | (4.6958) | (3.8629) | (4.2551) | (-3.5366) | (-1.5092) |
| Volatility 3 - 1 | Average Returns | 0.0190 | 0.0110 | 0.0107 | | |
| Volatility 3 - 1 | N-W t-stat | (3.1361) | (2.5827) | (2.8503) | | |
| Volatility 3 - 1 FFCPS alpha | Average Returns | 0.0116 | 0.0064 | 0.0117 | | |
| Volatility 3 - 1 FFCPS alpha | N-W t-stat | (2.6100) | (1.5438) | (2.6976) | | |

As for the size-volatility sort, the same trends as in the univariate sort case apply. Larger volatility and smaller size both lead to higher returns, respectively. **Most volatile, smallest of firms display the largest returns, and vice-versa.**

Interestingly enough, the FFCPS $\alpha$ effect is also present, more so in case of the size-differenced portfolio. To be specific, **the effect of size on returns nearly disappears** (mostly in case of medium volatile firms) **when controlled for other factors**. For volatility, this effect is less clear, and smaller in magnitude. However, the former discovery will surely deserve more scrutiny in the last section of this homework.

In tables 6 and 7, we then present the results of nearly the same procedure, the only difference being the calculation with one-quarter ahead returns, instead of one-month ahead.

<br>
<center><i><b>Table 6: Bivariate sort - size-beta - one-quarter ahead excess returns</b></i></center>

| Portfolio | Metrics | Size 1 | Size 2 | Size 3 | Size 3 - 1 | Size 3 - 1 FFCPS alpha |
|---|---|---|---|---|---|---|
| Beta 1 | Average Returns | 0.0175 | 0.0107 | 0.0102 | -0.0073 | -0.0057 |
| Beta 1 | N-W t-stat | (5.7410) | (4.2214) | (3.9922) | (-4.4529) | (-2.7248) |
| Beta 2 | Average Returns | 0.0213 | 0.0160 | 0.0123 | -0.009 | -0.0035 |
| Beta 2 | N-W t-stat | (4.4703) | (3.5544) | (3.3028) | (-3.493) | (-1.7662) |
| Beta 3 | Average Returns | 0.0323 | 0.0209 | 0.0168 | -0.0155 | -0.0087 |
| Beta 3 | N-W t-stat | (4.9121) | (3.7099) | (3.4919) | (-5.0641) | (-2.9797) |
| Beta 3 - 1 | Average Returns | 0.0148 | 0.0101 | 0.0066 | | |
| Beta 3 - 1 | N-W t-stat | (3.0465) | (2.3534) | (1.7404) | | |
| Beta 3 - 1 FFCPS alpha | Average Returns | 0.0098 | 0.0063 | 0.0068 | | |
| Beta 3 - 1 FFCPS alpha | N-W t-stat | (1.9128) | (1.0847) | (1.7641) | | |

<br>
<center><i><b>Table 7: Bivariate sort - size-volatility - one-quarter ahead excess returns</b></i></center>

| Portfolio | Metrics | Size 1 | Size 2 | Size 3 | Size 3 - 1 | Size 3 - 1 FFCPS alpha |
|---|---|---|---|---|---|---|
| Volatility 1 | Average Returns | 0.0143 | 0.0109 | 0.0100 | -0.0043 | -5e-04 |
| Volatility 1 | N-W t-stat | (4.9816) | (4.1472) | (3.4108) | (-2.2104) | (-0.1766) |
| Volatility 2 | Average Returns | 0.0219 | 0.0162 | 0.0130 | -0.0089 | -0.0065 |
| Volatility 2 | N-W t-stat | (4.9313) | (3.7347) | (3.6372) | (-4.1886) | (-2.6645) |
| Volatility 3 | Average Returns | 0.0305 | 0.0211 | 0.0192 | -0.0113 | -0.004 |
| Volatility 3 | N-W t-stat | (4.3240) | (3.9134) | (3.8226) | (-3.0762) | (-1.2267) |
| Volatility 3 - 1 | Average Returns | 0.0162 | 0.0102 | 0.0092 | | |
| Volatility 3 - 1 | N-W t-stat | (2.8977) | (2.5419) | (2.3515) | | |
| Volatility 3 - 1 FFCPS alpha | Average Returns | 0.0115 | 0.0069 | 0.0080 | | |
| Volatility 3 - 1 FFCPS alpha | N-W t-stat | (1.8561) | (1.3199) | (2.2265) | | |

When looking at one-quarter ahead excess returns instead, the general direction trends still remain, across both bivariate sorts. What can be noted, however, is that virtually all of returns are lower, across sorts and quantiles. This may be attributable to the fact that the market fluctuations are higher when observed at shorter time frame, but this is simply a speculation from our side.

Yet again, the differences in returns of the largest and smallest portfolio (for the respective sorting factors, of course) diminish, when controlling for the FFCPS factors. We feel like going into more details and particular cases for these would seem like digging for silver next to a gold vein, so we keep the picture of the overall trend in mind and move on to the last exercise, which should help us validate our suspicions.

#### Fama-MacBeth regressions

What we are looking for in the results of the last section of this homework is simple - we would like to examine the effect of volatility on returns, when being regressed on either alone, or along with other factors.

Table 8 displays the results of these estimations, 6 of them to be exact. Looking purely at the intercept, the highest returns are present in case no other factors (other than volatility) are added into the regression, while the lowest, in fact negative, returns appear when regressing additionally on illiquidity. But rather than the intercept alone, we should instead think of the interpretation as - if we take the first regression intercept as a benchmark, how does it change when other factors are added into the regression.

Firstly, it is interesting to note, that no intercept t-statistics are statistically significant at 5\% or less, apart from the third regression (on size). We could deduce from this, that **size has the single highest explanatory power**, when it comes to attributing the returns behavior to other factors.

Illiquidity, too, then has a highly statistically significant coefficient (in regression 5), meaning that it too holds explanatory power in the regression, only its coefficient alone is of negligible magnitude. We then also can not argue strongly for the behavior of the intercept in the same regression, as it is statistically insignificant.

Less important, but still a thing to note, are the signs of the coefficients of factors apart from volatility. The only two we can only comment with certain confidence on size and illiquidity (which are statistically significant). Both of these then possess the signs that theory would predict - negative for size (smaller firms have higher returns), and positive for  illiquidity (less liquid firms are traded less often, and are usually smaller, so higher returns).

Virtually all statistical significance then disappears in the regression of returns on all of the factors, so interpreting the coefficients seems unreasonable.

<br>
<center><i><b>Table 8: Fama-MacBeth regressions</b></i></center>

| Factor | Metric | 1 | 2 | 3 | 4 | 5 | 6 |
|---|---|---|---|---|---|---|---|
| Intercept | Coefficient | 0.0031 | 4e-04 | 0.011 | 0.0025 | -0.0027 | 0.0016 |
| Intercept | N-W t-stat | (0.5975) | (0.0805) | (2.4103) | (0.5258) | (-0.5964) | (0.1798) |
| Volatility | Coefficient | 0.0018 | 0.0028 | 0.0016 | 0.0012 | 0.0014 | 0.0015 |
| Volatility | N-W t-stat | (2.4255) | (4.2093) | (1.9874) | (1.101) | (2.0287) | (1.2205) |
| Beta | Coefficient | | -0.0065 | | | | 0.0001 |
| Beta | N-W t-stat | | (-1.2686) | | | | (0.0208) |
| Size | Coefficient | | | -0.0033 | | | -0.0024 |
| Size | N-W t-stat | | | (-4.7286) | | | (-0.9240) |
| Momentum | Coefficient | | | | 0 | | -0.0049 |
| Momentum | N-W t-stat | | | | (0.0029) | | (-0.4845) |
| Illiquidity | Coefficient | | | | | 0.0071 | 0.0040 |
| Illiquidity | N-W t-stat | | | | | (6.276) | (0.9605) |

## Conclusion

In this homework we took a look at how different factors play various roles at predicting future excess returns when it comes to portfolio sorting.

We conducted two types of portfolio sorts, once on a single factor, and once on two factors simultaneously, independently. The sorting factors were - beta, size, and volatility. We also tried to explain the return behavior using other factors, such as market excess returns, High-Minus-Low, Momentum, or Illiquidity using Fama-French-Carhart-Pastor-Stambaugh and Fama-MacBeth regressions.

The results we obtained were mostly in line with theory, where the firms with highest returns on average were those with high beta and volatility, as well as those with small size. The FFCPS regression also strongly suggested, that the discrepancies between the top and bottom quantile portfolios may be a bit inflated by an influence of other factors, which we listed above. We were then able to partially attribute this influence to the individual factors using the FMB regressions, although trends there left room for further conversation.

For the code used to perform all of the procedures, calculations, and various estimations, feel free to check the appendix below. Thank you for taking your time to read through our work, as it took us a good amount of effort and time to get it prepared in the form you see.

## Appendix

We provide the appendix in the form of a usable jupyter notebook, along with short markdown comments. The code runs quicker than the seminar code, as we tried to use vectors whenever possible (as long as it didn't lead to a hefty delay in production time).

### How to run:

1. Choose a **master folder** under an arbitrary path. The name of this folder does not matter.
2. In the master folder, create an empty folder and **put the script into a new folder**. **Name this new folder `scripts`**. Remember to mind the capitalization.
3. In the master folder, create a subfolder `data/` and **into this folder** put the following files *(all available in Moodle)*:
   1. `61505008_data_download.csv`
   2. `Asset_Pricing_HW_2_data.RData`
   3. `third_factor_rand.csv` 
4. Your master folder should look as follows:

```
|---scripts/
   |  HW2_Cala_Cechova_Krejcar.ipynb
|---data/
   |  61505008_data_download.csv
   |  Asset_Pricing_HW_2_data.RData
   |  third_factor_rand.csv
```

From here on, you should be able to run all cells without any issues. Make sure to keep the working directory in the master folder, as additional files (such as factor data) will be added in to the `data/` folder during running of the script.

In [3]:
# Required packages
packages <- c("stringr", "quantmod", "tseries", "dplyr", "xml2", "httr", "PortfolioAnalytics",
              "lubridate", "sandwich", "hash")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
  print(paste("Installing package ", packages[!installed_packages],"...", sep = ""))
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))
rm(list = ls()) #Clean environment
print('All packages loaded successfully...')

# Miscellaneous
options(repr.plot.width = 6, repr.plot.height = 5)
set.seed(420)

[1] "All packages loaded successfully..."


### Groundwork

Although we were able to obtain the data in the first homework, **we choose to use the provided dataset**, as this allows for easier and **more reliable reproducibility**. First we load the data, subset the necessary stocks, save these as new variables, and then discard the source data in order to alleviate some memory.

In [4]:
# Static variables
group_number <- 61505008
ticker_file <- "data/61505008_data_download.csv"
data_file <- "data/Asset_Pricing_HW_2_data.RData"
group_factor <- "data/third_factor_rand.csv"
data_files <- list(ticker_file, data_file, group_factor)

In [5]:
 # Set wd to project root
if (str_detect(getwd(), 'scripts')) {
    setwd('..')
}

# Assert presence of all required files
for (file in data_files) {
    if (!file.exists(file)) {
        print(paste0('File ', file, ' does not exist or has been misplaced.'))
    }
}

# Load tickers of required stocks
tickers <- as.character(unlist(read.csv(ticker_file)[2]))

# Check the assigned group factor
getFactor <- function(factor){
    data <- read.csv(factor)
    matching_row <- which(data[2] == group_number)
    assigned_factor <- as.character(data[matching_row, 3])
    print(paste0('The factor assigned to this group is ', assigned_factor, '.'))
}
getFactor(group_factor)

[1] "The factor assigned to this group is VOL."


While selecting only the desired tickers, we found that **a ticker that was assigned to our group was missing from the provided data**. We decided to remedy for this by writing a simple fix which replaces all of these missing assigned data by random data of a stock, that is provided in the source data set. As this was the case for only one stock, no trend bias should be introduced into the data.

In [6]:
# Load source data into the working directory
source_data <- load(data_file)

# Subset only for the stocks we will need
# Note - use these as BV$AAPL, not BV['AAPL']
BV <- book_value_sap500[tickers]
MKT <- MktCap_sap500[tickers]
OHLCV <- OHLCV_sap500[tickers]

# Assert presence of all assigned tickers in the source data - replace data of those that are missing
data_replacement_counter <- 1
for (ticker in tickers) {
    all_stock_names <- names(OHLCV)
    if (!ticker %in% all_stock_names) { # Missing file found
        print(paste0(ticker, ' is missing from the source data...'))
        missing_ticker_idx <- which(tickers == ticker) # Index of the missing file
        
        # Get replacement data
        while (names(book_value_sap500[data_replacement_counter]) %in% tickers) {
            data_replacement_counter <- data_replacement_counter + 1 # Search for a stock not from the assigned set
        }

        # Replace the data
        replacement_stock <- names(book_value_sap500)[data_replacement_counter] # Stock which shall serve as a replacement
        all_stock_names[missing_ticker_idx] <- replacement_stock # Fix names

        BV[missing_ticker_idx] <- book_value_sap500[replacement_stock]
        MKT[missing_ticker_idx] <- MktCap_sap500[replacement_stock]
        OHLCV[missing_ticker_idx] <- OHLCV_sap500[replacement_stock]

        names(BV) <- all_stock_names
        names(MKT) <- all_stock_names
        names(OHLCV) <- all_stock_names
        print(paste0('Data for ', ticker, ' replaced successfully with data from ', replacement_stock))

        data_replacement_counter <- data_replacement_counter + 1
    }
}

# Remove source data from working directory for better memory management
rm(list = source_data)

[1] "MOH is missing from the source data..."
[1] "Data for MOH replaced successfully with data from ABT"


### Data preprocessing

We define several functions to help us preprocess data.

In [7]:
######################################
### Data loading and preprocessing ###
######################################

# Download the desired factor and return it as a single xts object
#   factor_name(str) - Name of the factor to download. Accepts the following values:
#       -> 'FF3_daily', 'FF3', 'Mom', 'Illiq'
#   latest_expected_date(str) - Date which the dartmouth website currently provides as the latest
#
#   Note: If 'daily' is explicitly not defined in the factor_name, the downloaded factor will contain
#       monthly values
getFactor <- function(factor_name, latest_expected_date = '2022-03-31') {
    # Static variables #
    latest_expected_date <- as.Date(as.yearmon(latest_expected_date), frac = 1)

    # Path to the file on the website
    web_file_paths <- hash() 
    web_file_paths[['FF3_daily']] <- "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_TXT.zip"
    web_file_paths[['FF3']] <- "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip"
    web_file_paths[['Mom']] <- "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Momentum_Factor_TXT.zip"
    web_file_paths[['Illiq']] <- "https://faculty.chicagobooth.edu/-/media/faculty/lubos-pastor/data/liq_data_1962_2021.txt"

    # Check input validity
    if (!factor_name %in% keys(web_file_paths)) {
        stop('Invalid input specification. The function is not able to handle this input.')
    }

    # Path to where the file should be downloaded to and stored in
    folder_file_names <- hash()
    folder_file_names[['FF3_daily']] <- 'data/F-F_Research_Data_Factors_daily.zip'
    folder_file_names[['FF3']] <- 'data/F-F_Research_Data_Factors.zip'
    folder_file_names[['Mom']] <- 'data/F-F_Momentum_Factor.zip'
    folder_file_names[['Illiq']] <- 'data/Illiquidity_Factor.txt'

    # Column names to be extracted for each factor
    desired_cols <- hash()
    desired_cols[['FF3_daily']] <- c('t', 'Mkt-RF', 'SMB', 'HML', 'RF') 
    desired_cols[['FF3']] <- c('t', 'Mkt-RF', 'SMB', 'HML', 'RF') 
    desired_cols[['Mom']] <- c('t', 'Mom')
    desired_cols[['Illiq']] <- c('t', 'Agg Liq', 'Innov Liq', 'Traded Liq')

    # N of rows of each file to read
    desired_rows <- hash()
    desired_rows[['FF3_daily']] <- 25210
    desired_rows[['FF3']] <- 1149
    desired_rows[['Mom']] <- 1143
    desired_rows[['Illiq']] <- 713

    # Number of rows to be skipped
    skip_rows <- hash()
    skip_rows[['FF3_daily']] <- 5
    skip_rows[['FF3']] <- 4
    skip_rows[['Mom']] <- 14
    skip_rows[['Illiq']] <- 11

    # Data download, saving and loading
    web_file_path <- web_file_paths[[factor_name]]
    folder_file_path <- folder_file_names[[factor_name]]
    if (!file.exists(folder_file_path)){ # Download if doesn't exist
        download.file(web_file_path, destfile = folder_file_path)
    }
    if (grepl('.zip', folder_file_path, fixed=TRUE)){ # Unzip if file is .zip file
        unzip(folder_file_path, exdir = "data")
    }

    txt_path <- gsub('.zip', '.txt', folder_file_path) # Get .txt file path (data file)
    cols <- desired_cols[[factor_name]]
    rows <- desired_rows[[factor_name]]
    skip <- skip_rows[[factor_name]]

    factors_raw <- read.delim( # Read data into a var
        txt_path,
        col.names = cols,
        sep = '',
        nrows = rows,
        header = FALSE,
        skip = skip,
        stringsAsFactors = FALSE
    )
    
    date_chars <- as.character(factors_raw[['t']])
    if (!grepl('daily', factor_name, fixed = TRUE)){ # Adding a placeholder date for monthly data
        date_chars <- paste0(date_chars, '01')
    }
    factors_raw[['t']] <- as.Date(date_chars, '%Y%m%d') # Time index to date
    factors <- as.xts(factors_raw[, -1], order.by = factors_raw[['t']]) # To xts
    factors <- factors / 100
    if (!grepl('daily', factor_name, fixed = TRUE)){ # Get last day of month, if monthly index
        index(factors) <- as.Date(as.yearmon(index(factors)), frac = 1)
    }
    # Check if row subsetting is correct - Illiquidity provides different data, skip the check there
    latest_date_in_data <- tail(index(factors), n = 1)
    if(!latest_expected_date == latest_date_in_data & !factor_name == 'Illiq') {
        print('Latest date specified does not match the data last value. Check number of rows in the file.')
        print(paste0('The latest available index is ', latest_date_in_data))
    }
    if(factor_name == 'Illiq'){
        factors <- factors[,3] # Only last col
    }
    return(factors)
}

# Input an xts object with daily data of market capitalization and return this as an xts data frame
#   with monthly market capitalization.
getMonthlyMarketCap <- function(daily_market_cap_data) {
    stock_names <- names(daily_market_cap_data)
    daily_market_caps <- do.call('cbind', daily_market_cap_data)
    monthly_market_caps <- apply.monthly(daily_market_caps, tail, 1)
    colnames(monthly_market_caps) <- stock_names
    index(monthly_market_caps) <- as.Date(as.yearmon(index(monthly_market_caps)), frac = 1)
    return(monthly_market_caps)
}

# Input an xts object of monthly returns data and return a similar object of monthly momentum
getMonthlyMom <- function(monthly_returns_data){
    main_idx <- index(monthly_returns_data)
    monthly_mom <- list()
    for (i in 1:length(main_idx[-1])){
        end_date <- main_idx[i-1]
        start_date <- end_date %m-% months(10)
        dates <- paste0(start_date, '/', end_date)
        returns <- monthly_returns_data[dates]
        if (nrow(returns) < 11){ # Insufficient observations
            monthly_mom[[i]] <- NA
            next
        }
        mom <- lapply(returns, function(stock_returns){ # Iterate over stock returns
            return(prod(stock_returns + 1) - 1)
        })
        monthly_mom[[i]] <- unlist(mom)
    }
    monthly_mom <- as.xts(do.call('rbind', monthly_mom), order.by = main_idx[-1])
    colnames(monthly_mom) <- gsub('returns', 'mom', colnames(monthly_mom))
    index(monthly_mom) <- as.Date(as.yearmon(index(monthly_mom)), frac = 1)
    return(monthly_mom)
}

# Input the OHLCV data object for all stocks and return monthly illiquidities for all these stocks
getMonthlyIlliq <- function(daily_returns, monthly_returns, OHLCV_data){
    main_idx <- index(OHLCV_data)
    nom_ratios <- lapply(OHLCV_data, function(stock) { # Get the ratio for all stocks
        vol <- stock[,which(str_detect(colnames(stock), 'Volume'))]
        adj <- stock[,which(str_detect(colnames(stock), 'Adjusted'))]
        ratio <- (vol*adj)/1e10
        return(ratio)
    })
    nom_ratios <- do.call('cbind', nom_ratios)
    colnames(nom_ratios) <- colnames(daily_returns) # Unify indexes for easier computation
    abs_ret <- lapply(colnames(daily_returns), function(stock) {
        ret <- daily_returns[,stock]
        nom_rat <- nom_ratios[,stock]
        return(abs(ret) / nom_rat) # Ratio for one stock
    })
    abs_ret <- do.call('cbind', abs_ret)
    monthly_abs_ret <- apply.monthly(abs_ret, tail, 1)
    index(monthly_abs_ret) <- as.Date(as.yearmon(index(monthly_abs_ret)), frac = 1)

    # Compute monthly illiq
    ret_idx <- index(monthly_returns)
    monthly_illiq <- lapply(ret_idx, function(end_date) {
        start_date <- end_date %m-% months(12)
        dates <- paste0(start_date, '/', end_date)
        ratios <- monthly_abs_ret[dates]
        illiq <- lapply(ratios, function(ratio){
                return(log(1 + mean(ratio, na.rm = TRUE)))
        })
        return(unlist(illiq))
    })
    monthly_illiq_out <- as.xts(do.call('rbind', monthly_illiq), order.by = ret_idx)
    colnames(monthly_illiq_out) <- gsub('returns', 'illiq', colnames(monthly_illiq_out))
    index(monthly_illiq_out) <- as.Date(as.yearmon(index(monthly_illiq_out)), frac = 1)
    return(monthly_illiq_out)
}

# Input a list of OHLCV datasets and return a similar object, only subsetted for adjusted prices. Discard the rest of the columns
getAdjustedPrices <- function(OHLCV_data) {
    if (!class(OHLCV_data) == 'list') {
        print('The input data must be a list')
        break
    }
    yankPrices <- function(data) {
        adj_col_idx <- which(str_detect(colnames(data), 'Adjusted')) # Get index of column containing adjusted prices
        prices <- data[, adj_col_idx]
        return(prices)
    }
    data_out <- lapply(OHLCV_data, yankPrices)
    data_out <- do.call('cbind', data_out)
    return(data_out)
}

# Input an xts data frame containing raw adjusted prices and convert these to daily or monthly returns
getReturns <- function(adjusted_prices_data, type = 'monthly') {
    if (!type %in% c('daily', 'monthly')) {
        stop('Can only handle daily or monthly returns calculation...')
    }
    getReturn <- function(data) {
        if (type == 'monthly') {
            return(suppressWarnings(monthlyReturn(data)))
        } else {
            return(suppressWarnings(dailyReturn(data)))
        }
    }    
    returns <- lapply(adjusted_prices_data, getReturn) # Calculate returns for all stocks
    returns <- do.call('cbind', returns)
    colnames(returns) <- gsub('Adjusted', 'returns', names(adjusted_prices_data)) # Rename columns
    return(returns)
}

#########################################
### Performance indicator calculation ###
#########################################

# Input daily and monthly returns data, along with daily ff3 factors, all as an xts object.
#   Return an xts object containing monthly betas for the input, calculated using last 12 months of data.
getMonthlyBetas <- function(daily_returns_data, monthly_returns_data, daily_ff3_data) {
    names(monthly_returns_data) <- gsub('returns', 'betas', names(monthly_returns_data))
    stocks <- names(monthly_returns_data) # Stock names - AAPL.betas
    monthly_dates <- index(monthly_returns_data) # Dates by months

    monthly_betas <- lapply(monthly_dates, function(end_date) { #Betas for all
        start_date <- end_date %m-% months(12) # Last 12 months of data
        date_range <- paste0(start_date, '/', end_date)
        daily_returns <- daily_returns_data[date_range]
        market_returns <- daily_ff3_data$Mkt.RF[date_range]
        stocks_betas <- lapply(daily_returns, function(stock_returns) {
            model_data <- na.omit(cbind(stock_returns, market_returns))
            if (nrow(model_data) < 12 * 15) {
                return(NA)
            }
            model_betas <- lm(model_data[,1] ~ model_data[,2])
            return(model_betas$coefficients[[2]]) # Beta for given stock
        })
        return(unlist(stocks_betas)) # Betas for all stocks, given time frame
    })
    monthly_betas <- do.call('rbind', monthly_betas)
    monthly_betas <- as.xts(monthly_betas, order.by = monthly_dates)
    colnames(monthly_betas) <- stocks
    return(monthly_betas)
}

# Input a list of xts objects containing market capitalization info and convert these to monthly sizes
getMonthlySizes <- function(mkt_cap_data) {
    monthly_sizes <- lapply(mkt_cap_data, log) # Calculate monthly sizes
    monthly_sizes <- do.call('cbind', monthly_sizes)
    monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA))) # inf to NA
    colnames(monthly_sizes) <- paste0(colnames(monthly_sizes), '.sizes') # Rename columns
    return(monthly_sizes)
}

# Input two data frames (containing daily and monthly returns data) and return a data frame with monthly volatility
getMonthlyVolatilities <- function(daily_returns_data, monthly_returns_data) {
    names(monthly_returns_data) <- gsub('returns', 'volatility', names(monthly_returns_data))
    stocks <- names(monthly_returns_data) # Stock names - AAPL.returns
    monthly_dates <- index(monthly_returns_data) # Dates by months

    monthly_volatilities <- lapply(monthly_dates, function(end_date) { # Volatilities for all time frames, all stocks
        start_date <- end_date %m-% months(12) # Use last 12 months of data as suggested in the setup
        date_range <- paste0(start_date, '/', end_date)
        daily_returns <- daily_returns_data[date_range]
        stocks_volatilities <- lapply(daily_returns, function(stock) {
            average_stock_return <- mean(stock)
            nominator <- sum((stock - average_stock_return) ^ 2)
            denominator <- length(stock) - 1
            monthly_volatility <- 100 * sqrt(nominator/denominator) * sqrt(12)
            return(monthly_volatility) # Volatility for a given time frame, one stock
        })
        return(unlist(stocks_volatilities)) # Volatility for a given time frame, all stocks
    })
    monthly_volatilities <- do.call('rbind', monthly_volatilities) # Data frame of volatilities
    monthly_volatilities <- as.xts(monthly_volatilities, order.by = monthly_dates) # To xts
    colnames(monthly_volatilities) <- stocks
    return(monthly_volatilities)
}

In [8]:
# Compute the various metrics using previously defined functions
adjusted_prices <- getAdjustedPrices(OHLCV)
daily_returns <- getReturns(adjusted_prices, type = 'daily')
monthly_returns <- getReturns(adjusted_prices, type = 'monthly')
monthly_market_caps <- getMonthlyMarketCap(MKT)
# Download factors from online webistes and transform them into a pretty xts
daily_ff3 <- getFactor('FF3_daily')
monthly_ff3 <- getFactor('FF3')
# There are two versions of data for momentum and illiquidity - one to be used in the
#    Fama-French-Carhart-Pastor-Stambaugh model, and one for the Fama-MacBeth regression
monthly_mom_ffcps <- getFactor('Mom') # Momentum factor - used in FFCPS model
monthly_illiq_ffcps <- getFactor('Illiq') # Illiquidity facotr - used in FFCPS model
monthly_mom_fmb <- getMonthlyMom(monthly_returns) # Momentum factor - used in FMB regression
monthly_illiq_fmb <- getMonthlyIlliq(daily_returns, monthly_returns, OHLCV) # Illiq. factor - FMB regression

In [9]:
# Compute the three desired factors (monthly)
monthly_sizes <- getMonthlySizes(monthly_market_caps) # 5s
monthly_betas <- getMonthlyBetas(daily_returns, monthly_returns, daily_ff3) # 2 mins
monthly_volatilities <- getMonthlyVolatilities(daily_returns, monthly_returns) # Monthly volatilities (vectors go brrrrrrr) - 9s

### Univariate portfolio analysis

In [10]:
# Input monthly data to sort by, factor name, monthly returns data, and monthly market capital data.
#   Furthermore specify the number of quantiles and the method by which to compute the returns.
#   For this, two options are available - 'equal' (as in equally weighted) and 'value' (as in
#       value weighted).
#   Return overall average returns within portfolios and the corresponding standard errors
#       as a simple data frame.
#   If 'get_quintile_values' == TRUE, return quintile values for this factor instead.
getUnivariateSort <- function(sort_variable_data, factor_name, monthly_returns_data,
                                monthly_market_caps, monthly_ff3_data, monthly_mom_data,
                                monthly_illiq_data, n = 5, method = 'equal',
                                get_quintile_values = FALSE){
    # Static
    diff_portfolio <- paste0(n, ' - 1')
    portfolio_colnames <- c(1:n, diff_portfolio)
    portfolio_returns_index <- index(sort_variable_data)[-1] # All but first element

    # Iterate over the months in the portfolio returns index and return return for each of them
    portfolio_returns <- lapply(portfolio_returns_index, function(current_month) {
        this_month_sort_variable <- sort_variable_data[current_month]
        not_na <- !is.na(this_month_sort_variable)
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)
        
        # Check for data validity, return a vector of NAs in case of invalid data
        skip_conditions <- c(
            sum(not_na) < 4*n, # Not enough observations for this month
            !next_month %in% index(monthly_returns_data) # No returns data in the next month

        )
        if (method == 'value') { # Skip conditions that apply only for the value weighted method
            # No market capitalization data for the next month - nothing to weigh by
            skip_conditions <- append(skip_conditions, !next_month %in% index(monthly_market_caps)) 
        }
        if (any(skip_conditions)) {
            vec_length <- ifelse (get_quintile_values, n, n+1) # No diff in getting quintile values
            return(c(rep(NA, vec_length))) # Return vector of NAs
        }

        next_month_returns <- monthly_returns_data[next_month] # A vector for all stocks
        next_month_market_caps <- monthly_market_caps[next_month]
        next_month_market_caps[is.na(next_month_market_caps)] <- 0 # No weights for missing obs

        breakpoints <- quantile(this_month_sort_variable, prob = 0:n/n, na.rm = TRUE) # Quintiles

        # Compute portfolio returns
        this_period_returns <- c()
        this_period_quintile_averages <- c() # For storing quintile values
        for (i in breakpoints) { # Find all quintile breakpoints
            next_breakpoint_idx <- match(i, breakpoints) + 1
            next_breakpoint <- breakpoints[next_breakpoint_idx]
            if (!is.na(next_breakpoint)) { # While not out of quntiles
                filter <- (i < this_month_sort_variable) &
                          (this_month_sort_variable < next_breakpoint) &
                          (not_na)
                if (method == 'equal') {# Equally weighted portfolio
                    quintile_return <- mean(next_month_returns[,filter])
                } else if (method == 'value') {
                    quintile_return <- weighted.mean(t(next_month_returns[,filter]),
                                                    t(next_month_market_caps[,filter])) # Actual return
                } else {
                    stop('Incorrectly defined method. Select one of - equal, value.')
                }
                if (get_quintile_values) { # Store average factor value in this quintile
                    this_period_quintile_averages <- append(this_period_quintile_averages,
                        mean(this_month_sort_variable[,filter]))
                }
                this_period_returns <- append(this_period_returns, quintile_return) # Store value 
            } else {
                break # Last iteration
            } 
        }
        if (get_quintile_values) {
            return(this_period_quintile_averages) # Return quintile averages instead
        }
        diff_returns <- this_period_returns[[n]] - this_period_returns[[1]]
        this_period_returns <- append(this_period_returns, diff_returns) # Append returns of diff
        return(this_period_returns) # Returns for all quintiles for this period
    })
    portfolio_returns <- as.xts(do.call('rbind', portfolio_returns), # To xts
        order.by = as.Date(as.yearmon(portfolio_returns_index), frac = 1))
    
    if (get_quintile_values){ # Calculate quintile values instead
        quintile_values <- lapply(portfolio_returns, function(quintile) {
            return(mean(quintile, na.rm = TRUE))
        })
        quintile_values <- as.data.frame(do.call('cbind', quintile_values))
        data_out <- cbind(factor_name, quintile_values)
        colnames(data_out) <- c('Factor', 1:n)
        return(data_out) # Actually quintile averages
    }

    # Use the monthly portfolio returns to calculate avg returns and their standard errors
    average_returns <- lapply(portfolio_returns, function(col) {
        model_data <- cbind(na.omit(col), 1)
        colnames(model_data) <- c('returns', 'one')
        model <- lm(formula = returns ~ one, data = model_data) # Explicit to avoid package error
        avg_ret <- model$coefficients[[1]]
        std_err <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
        return(c(avg_ret, std_err))
    })

    # Calculate FF alpha and the corresponding NeweyWest t-statistic
    diff_returns <- portfolio_returns[,n + 1] # Returns of the diff portfolio
    diff_returns <- diff_returns[!is.na(diff_returns)] # Remove NAs
    diff_idx <- index(diff_returns)
    ff_data <- monthly_ff3_data[diff_idx, c('Mkt.RF', 'SMB', 'HML')]
    mom_data <- monthly_mom_data[diff_idx]
    illiq_data <- monthly_illiq_data[diff_idx]
    ff_lm_data <- cbind(diff_returns, ff_data, mom_data, illiq_data)
    colnames(ff_lm_data) <- c('diff_returns', 'Mkt.RF', 'SMB', 'HML', 'Mom', 'Illiq')
    ff_model <- lm(diff_returns ~ Mkt.RF + SMB + HML + Mom + Illiq, data = ff_lm_data) # FF3 regression
    ff_alpha <- ff_model$coefficients[[1]]
    ff_alpha_t <- suppressWarnings(ff_model$coefficients[[1]] / sqrt(NeweyWest(ff_model,
                    lag = 6))[[1]])
    ff <- c(ff_alpha, ff_alpha_t)

    # Output data frame construction
    metrics <- c('Average Returns', 'NeweyWest t-stat')
    average_returns <- as.data.frame(do.call('cbind', average_returns)) # To data frame
    data_out <- cbind(factor_name, metrics, average_returns, ff)
    colnames(data_out) <- c('Factor', 'Metric', portfolio_colnames, 'FFCPS alpha')
    return(data_out)
}

# Specify the input data and return a prettified output as a data frame, in a fashion similar to the
#   table from lecture 10
#   input_data(list) - A list of the input data to be sorted by
#   factor_names(list) - A list of names of the factors to be used
#   Note: The rest of the arguments are similar as in the previous functions
getVariateSortOutput <- function(input_data, factor_names, monthly_returns_data,
        monthly_market_caps_data, monthly_ff3_data, monthly_mom_data,
        monthly_illiq_data, n = 5, method = 'equal', get_quintile_values = FALSE) {
    input_length <- length(input_data)
    if (!input_length == length(factor_names)) {
        stop('The input data length must be the same as their names.')
    }
    data_out <- list()
    for (i in 1:input_length) { # Generate the portfolio sort for all factors
        input_sort <- input_data[[i]]
        input_factor <- factor_names[[i]]
        sort_results <- getUnivariateSort(input_sort, input_factor, monthly_returns_data,
            monthly_market_caps_data, monthly_ff3_data, monthly_mom_data,
            monthly_illiq_data, n = n, method = method, get_quintile_values = get_quintile_values)
        data_out <- rbind(data_out, sort_results)
    }
    start_round_col <- ifelse(get_quintile_values, 2, 3)
    for(i in 1:nrow(data_out)){
        for(j in start_round_col:ncol(data_out)){
            data_out[i,j] <- round(data_out[i,j], 4) # Round all numbers
        }
    }
    return(data_out)
}

In [11]:
# Univariate sort calculation

# Static variables
sort_output_input_data <- list(monthly_betas, monthly_sizes, monthly_volatilities)
sort_output_factor_names <- list('Beta', 'Size', 'Volatility')

# Quintile values
quintile_values <- getVariateSortOutput(sort_output_input_data, sort_output_factor_names,
    monthly_returns, monthly_market_caps, monthly_ff3, monthly_mom_ffcps,
    monthly_illiq_ffcps, get_quintile_values = TRUE)

# Equally weighted univariate sort
equal_weighted_uni_sort <- getVariateSortOutput(sort_output_input_data, sort_output_factor_names,
    monthly_returns, monthly_market_caps, monthly_ff3, monthly_mom_ffcps,
    monthly_illiq_ffcps, n = 5, method = 'equal')

# Value weighted univariate sort
value_weighted_uni_sort <- getVariateSortOutput(sort_output_input_data, sort_output_factor_names,
    monthly_returns, monthly_market_caps, monthly_ff3, monthly_mom_ffcps,
    monthly_illiq_ffcps, n = 5, method = 'value')

In [12]:
print('Quintile average values:')
quintile_values

print('Equally weighted univariate sort:')
equal_weighted_uni_sort

print('Value weighted univariate sort:')
value_weighted_uni_sort

[1] "Quintile average values:"


Factor     1      2      3      4      5      
1 Beta       0.4858 0.7563 0.9418 1.1482  1.5955
2 Size       1.4119 2.1240 2.6941 3.3453  4.5757
3 Volatility 4.3400 5.5260 6.4991 7.8382 11.2847

[1] "Equally weighted univariate sort:"


Factor     Metric           1      2      3      4      5      5 - 1  
1 Beta       Average Returns  0.0102 0.0125 0.0133 0.0169 0.0174  0.0072
2 Beta       NeweyWest t-stat 4.5661 3.9316 3.5691 3.6490 2.9726  1.5407
3 Size       Average Returns  0.0258 0.0144 0.0129 0.0109 0.0099 -0.0160
4 Size       NeweyWest t-stat 4.8098 3.2872 2.7436 3.1781 2.7838 -5.1373
5 Volatility Average Returns  0.0104 0.0126 0.0137 0.0163 0.0205  0.0101
6 Volatility NeweyWest t-stat 4.7774 4.2211 3.7720 3.8077 3.5090  2.2479
  FFCPS alpha
1  0.0076    
2  1.5634    
3 -0.0157    
4 -5.2873    
5  0.0106    
6  2.1967

[1] "Value weighted univariate sort:"


Factor     Metric           1      2      3      4      5      5 - 1  
1 Beta       Average Returns  0.0109 0.0112 0.0109 0.0159 0.0215  0.0107
2 Beta       NeweyWest t-stat 4.3326 3.8501 2.9751 3.4958 3.8753  2.0778
3 Size       Average Returns  0.0311 0.0199 0.0176 0.0146 0.0126 -0.0185
4 Size       NeweyWest t-stat 5.1604 4.4502 3.8404 4.3147 3.9384 -4.6305
5 Volatility Average Returns  0.0090 0.0111 0.0151 0.0159 0.0249  0.0159
6 Volatility NeweyWest t-stat 3.7261 3.2137 4.2642 3.6027 4.4187  3.2809
  FFCPS alpha
1  0.0109    
2  1.9444    
3 -0.0186    
4 -5.1836    
5  0.0171    
6  3.3304

### Bivariate portfolio analysis

In [13]:
# Specify the characteristics and perform an independent bivariate portfolio sort
#   on two factors. Supports only value-weighted sort.
# Return a table with average returns with a corresponding Newey-West t-stat for all
#   quantiles, a difference portfolio, and lastly a FF-alpha with its t-stat.
# The arguments are similar to the univariate sort, only two factors are provided
#   instead of one. 'months_ahead' then specifies the #months to look-ahead for returns.
getBivariateSort <- function(first_sort_data, first_factor_name, second_sort_data,
                second_factor_name, monthly_returns_data, monthly_market_caps_data,
                monthly_ff3_data, monthly_mom_data, monthly_illiq_data, n = 3, months_ahead = 1){
    # Static
    diff_portfolio <- paste0(n, ' - 1')
    bps <- c(0, 0.3, 0.7, 1)
    if (!length(bps) == (n + 1)) {
        stop('Incorrectly specified portfolio breakpoints')
    }

    # Match indexes
    index(first_sort_data) <- as.Date(as.yearmon(index(first_sort_data)), frac = 1)
    index(second_sort_data) <- as.Date(as.yearmon(index(second_sort_data)), frac = 1)
    factor_indexes_match <- index(first_sort_data) %in% index(second_sort_data)
    main_idx <- index(first_sort_data[factor_indexes_match][-1]) # Skip first observation
    #Subset the sort data only to the observations where both values are available
    first_sort_data <- first_sort_data[main_idx]
    second_sort_data <- second_sort_data[main_idx]

    # Get portfolio returns for all portfolios
    portfolio_returns <- lapply(main_idx, function(current_month) {
        next_month <- as.Date(as.yearmon(current_month %m+% months(months_ahead)), frac = 1)
        if (!next_month %in% index(monthly_returns_data)){ 
            return(NA) # No monthly returns in this period
        }
        
        # Current month variables
        month_first_factor <- first_sort_data[current_month]
        month_second_factor <- second_sort_data[current_month]
        first_not_na <- !is.na(month_first_factor)
        second_not_na <- !is.na(month_second_factor)
        
        #Next month variables
        month_returns <- monthly_returns_data[next_month]
        month_market_caps <- monthly_market_caps_data[next_month]
        month_market_caps[is.na(month_market_caps)] <- 0
        
        # Skip in case of invalid data
        skip_conditions <- c(
            !any(first_not_na), # No data on first factor
            !any(second_not_na), # No data on second factor
            sum(month_market_caps) == 0 # No data on market capitalization
        )
        if (any(skip_conditions)) {
            return(NA) # Skip this month
        }

        # Find breakpoints
        first_breakpoints <- quantile(month_first_factor, bps, na.rm = TRUE)
        second_breakpoints <- quantile(month_second_factor, bps, na.rm = TRUE)

        # Create all combinations of the two breakpoints for the independent sort - nested list
        tuple_list <- apply(expand.grid(first_breakpoints, second_breakpoints), 1, list)
        current_month_out <- lapply(tuple_list, function(y) { # Iterate over all tuples
            first_bp_1 <- y[[1]][[1]] # 1st factor starting breakpoint
            second_bp_1 <- y[[1]][[2]] # 2nd factor staring breakpoint

            next_bp_idx_1 <- match(first_bp_1, first_breakpoints) + 1
            next_bp_idx_2 <- match(second_bp_1, second_breakpoints) + 1

            first_bp_2 <- first_breakpoints[next_bp_idx_1] # 1st factor ending breakpoint
            second_bp_2 <- second_breakpoints[next_bp_idx_2] # 2nd factor ending breakpoint

            if (is.na(first_bp_2) | is.na(second_bp_2)) { # Out of quantiles
                return('ooq')
            }

            #Construct the filters
            first_filter <- (month_first_factor < first_bp_2) &
                            (month_first_factor > first_bp_1) &
                            first_not_na
            second_filter <- (month_second_factor < second_bp_2) &
                             (month_second_factor > second_bp_1) &
                             second_not_na
            merged_filter <- (first_filter) & (second_filter)

            # Get returns for current month's weighted portfolio
            filtered_returns <- month_returns[, merged_filter]
            filtered_caps <- month_market_caps[,merged_filter]
            current_returns <- weighted.mean(t(filtered_returns), t(filtered_caps))
            return(current_returns)
        })
        current_month_out <- current_month_out[!current_month_out == 'ooq'] # Remove out of index quantiles
        current_month_out <- do.call('cbind', current_month_out)
        return(current_month_out)
    })
    portfolio_returns <- as.data.frame(do.call('rbind', portfolio_returns))
    rownames(portfolio_returns) <- main_idx
    tuple_grid <- expand.grid(c(1:n), c(1:n)) # Grid of all combinations of portfolio name tuples
    tuple_colnames <- sprintf('(%s,%s)', tuple_grid[,2], tuple_grid[,1]) # Coordinates ! (x,y)
    colnames(portfolio_returns) <- tuple_colnames 

    # Add returns of diff portfolios
    diff_cols <- c() # Diff portfolio column names

    for (i in 1:n) { # First factor diff portfolios
        temp_col <- paste0('first_diff_', i)
        diff_cols <- append(diff_cols, temp_col) # Store column name
        last_idx <- paste0('(', i, ',', n, ')') # Such as (1,3) - coordinates of matrix ! (x,y)
        first_idx <- paste0('(', i, ',' , 1, ')') # Such as (1,1)
        temp_returns <- portfolio_returns[,last_idx] - portfolio_returns[,first_idx]
        portfolio_returns[,temp_col] <- temp_returns # Assign column with diff portfolio returns
    }

    for (i in 1:n) { # Second factor diff portfolios
        temp_col <- paste0('second_diff_', i)
        diff_cols <- append(diff_cols, temp_col) # Store column name
        last_idx <- paste0('(', n, ',', i, ')') # Such as (3,1)
        first_idx <- paste0('(', 1, ',' , i, ')') 
        temp_returns <- portfolio_returns[,last_idx] - portfolio_returns[,first_idx]
        portfolio_returns[,temp_col] <- temp_returns # Assign column with diff portfolio returns
    }

    # Calculate average returns and newey-test stat for all portfolios (including diff)
    average_returns <- lapply(portfolio_returns, function(col) {
        model_data <- as.data.frame(cbind(na.omit(col), 1))
        colnames(model_data) <- c('returns', 'one')
        model <- lm(formula = returns ~ one, data = model_data) # Explicit to avoid package error
        avg_ret <- model$coefficients[[1]]
        std_err <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
        return(c(avg_ret, std_err))
    })

    # Calculate FF3 alphas for both factors
    ff_alphas <- lapply(diff_cols, function(col){
        diff_returns <- portfolio_returns[,col] # Returns for this diff portfolio
        diff_returns <- diff_returns[!is.na(diff_returns)] # Remove NAs
        diff_idx <- index(diff_returns)
        ff_data <- monthly_ff3_data[diff_idx, c('Mkt.RF', 'SMB', 'HML')]
        ff_lm_data <- as.data.frame(cbind(diff_returns, ff_data))
        colnames(ff_lm_data) <- c('diff_returns', 'Mkt.RF', 'SMB', 'HML')
        ff_lm_data$Mom <- monthly_mom_data[diff_idx,] # Explicit col assignment to avoid errors
        ff_lm_data$Illiq <- monthly_illiq_data[diff_idx,]
        ff_model <- lm(diff_returns ~ Mkt.RF + SMB + HML + Mom + Illiq, data = ff_lm_data) # FF3 regression
        ff_alpha <- ff_model$coefficients[[1]]
        ff_alpha_t <- suppressWarnings(ff_model$coefficients[[1]] / sqrt(NeweyWest(ff_model,
                        lag = 6))[[1]])
        ff <- c(ff_alpha, ff_alpha_t)
        return(ff)
    })
    for (i in 1:(2*n)) { # Assign these to the average_returns object
        factor_colname <- paste0(diff_cols[i], '_ff_alpha')
        average_returns[factor_colname] <- ff_alphas[i]
    }
    
    # Averages and Newey-test t-stats of all portolios to a data frame
    all_returns <- lapply(average_returns, function(y){y[1]}) # Only returns
    all_nw <- lapply(average_returns, function(y){y[2]}) # Only N-W t-stats

    # Create and populate an output matrix/data frame
    temp_out <- matrix(nrow = n * 2 + 2 * 2, ncol = n + 2) # An empty matrix to store values
    fillTemp <- function(temp, rets, nw, dim, n) {
        it_rows <- n + 2 # Number of rows to iterate over
        for (i in 1:it_rows) { # Iterate over all rows
            in_quintile <- (i <= n)
            in_diff <- (n < i) & (i <= n + 1)
            if (in_quintile) { # Main quantiles
                ass_idx <- (i - 1) * 2 + 1 # Index of the row to assign to
                for (j in 1:n) { # Fill main quantiles
                    get_idx <- (i - 1) * 3 + j # Index of this metric in all_returns/all_nw
                    temp[ass_idx,j] <- rets[[get_idx]][1] # Return
                    temp[ass_idx + 1, j] <- nw[[get_idx]][1] # t-stat
                }
                temp[ass_idx, n + 1] <- rets[[n^2 + i]][1] # Difference portfolio returns (vert)
                temp[ass_idx + 1, n + 1] <- nw[[n^2 + i]][1] # Difference portfolio t-stat (vert)
                temp[ass_idx, n + 2] <- rets[[n^2 + 2 * n + i]][1] # FF alpha returns (vert)
                temp[ass_idx + 1, n + 2] <- nw[[n^2 + 2 * n + i]][1] # FF alpha t-stat (vert)
            } else if (in_diff){ # Diff portfolio
                ass_idx <- n * 2 + 1 # Index of the row to assign to
                for (j in 1:n) { # Across all quantiles
                    get_idx <- n^2 + n + j
                    temp[ass_idx, j] <- rets[[get_idx]][1] # Difference portfolio returns (hor)
                    temp[ass_idx + 1, j] <- nw[[get_idx]][1] # Difference portolio t-stat (hor)
                }
            } else { # FF alpha
                ass_idx <- n * 2 + 3
                for (j in 1:n) { # Across all quantiles
                    get_idx <- n^2 + 3 * n + j
                    temp[ass_idx, j] <- rets[[get_idx]][1] # FF alpha returns (hor)
                    temp[ass_idx + 1, j] <- nw[[get_idx]][1] # FF alpha t-stat (hor)
                }
            }
        }
        for(i in 1:nrow(temp)){
            for(j in 1:ncol(temp)){
                temp[i,j] <- round(temp[i,j], 4) # Round all numbers
            }
        }
        return(temp)
    }
    data_out <- fillTemp(temp_out, all_returns, all_nw, matrix_dim, n)
    # Output handling and transformation
    metrics <- rep(c('Average Returns', 'N-W t-stat'), n + 2)
    pf_names <- rep(c(paste(second_factor_name, 1:n), paste(second_factor_name, n, '- 1'),
                      paste(second_factor_name, n, '- 1 FFCPS alpha')),
                    c(rep(2, n + 2))) # Repeat all elements up to n + 2
    data_out <- as.data.frame(data_out)
    data_out[is.na(data_out)] <- " " # NAs to empty strings for pretty output
    data_out <- cbind(pf_names, metrics, data_out)
    data_cols <- c('Portfolio', 'Metrics',
                    paste(first_factor_name, 1:n), paste(first_factor_name, n, ' - 1'),
                    paste(first_factor_name, n, '- 1 FFCPS alpha'))
    colnames(data_out) <- data_cols
    return(data_out)
    }

In [14]:
# Bivariate sort calculation
bivariate_sort_size_beta_1m <- getBivariateSort(monthly_sizes, 'Size', monthly_betas, 'Beta',
    monthly_returns, monthly_market_caps, monthly_ff3, monthly_mom_ffcps,
    monthly_illiq_ffcps, months_ahead = 1)

bivariate_sort_size_vol_1m <- getBivariateSort(monthly_sizes, 'Size', monthly_volatilities, 'Volatility',
    monthly_returns, monthly_market_caps, monthly_ff3, monthly_mom_ffcps,
    monthly_illiq_ffcps, months_ahead = 1)

bivariate_sort_size_beta_3m <- getBivariateSort(monthly_sizes, 'Size', monthly_betas, 'Beta',
    monthly_returns, monthly_market_caps, monthly_ff3, monthly_mom_ffcps,
    monthly_illiq_ffcps, months_ahead = 3)

bivariate_sort_size_vol_3m <- getBivariateSort(monthly_sizes, 'Size', monthly_volatilities, 'Volatility',
    monthly_returns, monthly_market_caps, monthly_ff3, monthly_mom_ffcps,
    monthly_illiq_ffcps, months_ahead = 3)

In [15]:
print('Value-weighted independent bivariate portfolio sort on Size-Beta - one-month-ahead:')
bivariate_sort_size_beta_1m

print('Value-weighted independent bivariate portfolio sort on Size-Volatility - one-month-ahead:')
bivariate_sort_size_vol_1m

print('Value-weighted independent bivariate portfolio sort on Size-Beta - one-quarter-ahead:')
bivariate_sort_size_beta_3m

print('Value-weighted independent bivariate portfolio sort on Size-Volatility - one-quarter-ahead:')
bivariate_sort_size_vol_3m

[1] "Value-weighted independent bivariate portfolio sort on Size-Beta - one-month-ahead:"


Portfolio              Metrics         Size 1 Size 2 Size 3 Size 3  - 1
1  Beta 1                 Average Returns 0.0188 0.0112 0.0110 -0.0078    
2  Beta 1                 N-W t-stat      5.8663 4.2603 4.7210 -4.1704    
3  Beta 2                 Average Returns 0.0226 0.0151 0.0120 -0.0106    
4  Beta 2                 N-W t-stat      4.5934 3.5177 3.1402 -4.2714    
5  Beta 3                 Average Returns 0.0360 0.0214 0.0176 -0.0184    
6  Beta 3                 N-W t-stat      4.9506 3.6224 3.6945 -4.5138    
7  Beta 3 - 1             Average Returns 0.0172 0.0102 0.0066            
8  Beta 3 - 1             N-W t-stat      2.9621 2.2228 1.6235            
9  Beta 3 - 1 FFCPS alpha Average Returns 0.0114 0.0061 0.0101            
10 Beta 3 - 1 FFCPS alpha N-W t-stat      2.2462 1.2058 2.1869            
   Size 3 - 1 FFCPS alpha
1  -0.0072               
2  -3.551                
3  -0.0068               
4  -3.2791               
5  -0.0085               
6  -2.304                
7                        
8                        
9                        
10

[1] "Value-weighted independent bivariate portfolio sort on Size-Volatility - one-month-ahead:"


Portfolio                    Metrics         Size 1 Size 2 Size 3
1  Volatility 1                 Average Returns 0.0164 0.0112 0.0100
2  Volatility 1                 N-W t-stat      5.7918 4.2531 3.6592
3  Volatility 2                 Average Returns 0.0225 0.0142 0.0133
4  Volatility 2                 N-W t-stat      4.8468 3.3325 3.6714
5  Volatility 3                 Average Returns 0.0353 0.0223 0.0207
6  Volatility 3                 N-W t-stat      4.6958 3.8629 4.2551
7  Volatility 3 - 1             Average Returns 0.0190 0.0110 0.0107
8  Volatility 3 - 1             N-W t-stat      3.1361 2.5827 2.8503
9  Volatility 3 - 1 FFCPS alpha Average Returns 0.0116 0.0064 0.0117
10 Volatility 3 - 1 FFCPS alpha N-W t-stat      2.6100 1.5438 2.6976
   Size 3  - 1 Size 3 - 1 FFCPS alpha
1  -0.0064     -0.0052               
2  -4.7271     -3.045                
3  -0.0091     -0.007                
4  -3.6964     -3.2542               
5  -0.0147     -0.0052               
6  -3.5366     -1.5092               
7                                    
8                                    
9                                    
10

[1] "Value-weighted independent bivariate portfolio sort on Size-Beta - one-quarter-ahead:"


Portfolio              Metrics         Size 1 Size 2 Size 3 Size 3  - 1
1  Beta 1                 Average Returns 0.0175 0.0107 0.0102 -0.0073    
2  Beta 1                 N-W t-stat      5.7410 4.2214 3.9922 -4.4529    
3  Beta 2                 Average Returns 0.0213 0.0160 0.0123 -0.009     
4  Beta 2                 N-W t-stat      4.4703 3.5544 3.3028 -3.493     
5  Beta 3                 Average Returns 0.0323 0.0209 0.0168 -0.0155    
6  Beta 3                 N-W t-stat      4.9121 3.7099 3.4919 -5.0641    
7  Beta 3 - 1             Average Returns 0.0148 0.0101 0.0066            
8  Beta 3 - 1             N-W t-stat      3.0465 2.3534 1.7404            
9  Beta 3 - 1 FFCPS alpha Average Returns 0.0098 0.0063 0.0068            
10 Beta 3 - 1 FFCPS alpha N-W t-stat      1.9128 1.0847 1.7641            
   Size 3 - 1 FFCPS alpha
1  -0.0057               
2  -2.7248               
3  -0.0035               
4  -1.7662               
5  -0.0087               
6  -2.9797               
7                        
8                        
9                        
10

[1] "Value-weighted independent bivariate portfolio sort on Size-Volatility - one-quarter-ahead:"


Portfolio                    Metrics         Size 1 Size 2 Size 3
1  Volatility 1                 Average Returns 0.0143 0.0109 0.0100
2  Volatility 1                 N-W t-stat      4.9816 4.1472 3.4108
3  Volatility 2                 Average Returns 0.0219 0.0162 0.0130
4  Volatility 2                 N-W t-stat      4.9313 3.7347 3.6372
5  Volatility 3                 Average Returns 0.0305 0.0211 0.0192
6  Volatility 3                 N-W t-stat      4.3240 3.9134 3.8226
7  Volatility 3 - 1             Average Returns 0.0162 0.0102 0.0092
8  Volatility 3 - 1             N-W t-stat      2.8977 2.5419 2.3515
9  Volatility 3 - 1 FFCPS alpha Average Returns 0.0115 0.0069 0.0080
10 Volatility 3 - 1 FFCPS alpha N-W t-stat      1.8561 1.3199 2.2265
   Size 3  - 1 Size 3 - 1 FFCPS alpha
1  -0.0043     -5e-04                
2  -2.2104     -0.1766               
3  -0.0089     -0.0065               
4  -4.1886     -2.6645               
5  -0.0113     -0.004                
6  -3.0762     -1.2267               
7                                    
8                                    
9                                    
10

### Fama-MacBeth regression

In [16]:
# Define the explained data and the explanatory data and perform a Fama-Mac Beth regression
#   explained_data(xts) - Explained data in the regression - usually monthly returns
#   explanatory_data(list) - Explanatory data in the regression - a list of xts objects
#   explanatory_data_names(vector) - A vector with names of factors in the explanatory_data list
getFMBRegression <- function(explained_data, explanatory_data, explanatory_data_names) {
    # Static variables
    names(explanatory_data) <- explanatory_data_names
    output_names <- c('Returns', explanatory_data_names)
    main_idx <- as.Date(as.yearmon(index(explained_data)), frac = 1) # Index of returns data

    # Match indices - all indexes of explanatory data should be found in the index of
    #    the explained data
    explanatory_data <- lapply(explanatory_data, function(data) {
        paired_idx <- as.Date(as.yearmon(index(data)), frac = 1)
        indexes_match <- paired_idx %in% main_idx
        return(data[indexes_match]) # Only matching values
    })

    fmb_coefs <- lapply(main_idx[-1], function(current_month){ # 1st month skipped
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)
        skip_conditions <- c(
            all(is.na(explained_data[next_month])) # No monthly returns in next period
            #all(is.na(explanatory_data[current_month])) # No explanatory data
        )
        if (any(skip_conditions)) {
            return(NA) # Skip this month
        }
        month_explained <- explained_data[next_month] # Next month returns (ind var)
        month_explanatory <- lapply(explanatory_data, function(data){ # Iterate over all explanat vars
            month_expl <- data[current_month]
            if (all(is.na(month_expl))){
                return(NA)
            }
            month_expl[which(month_expl == -Inf)] = NA
            month_expl[which(month_expl == Inf)] = NA
            if (all(is.na(month_expl))) {
                return(NA) # Remaining values were only +- infinity
            }
            return(t(month_expl))
        })
        transformed_explanatory <- as.data.frame(do.call('cbind', month_explanatory))
        transformed_explained <- t(month_explained)
        model_data <- na.omit(as.data.frame(cbind(transformed_explained, transformed_explanatory)))
        if (all(is.na(model_data))){ # An empty data frame
            return(NA)
        }
        colnames(model_data) <- output_names
        model_formula <- paste(output_names[1], '~ 1 +', paste0(output_names[-1], collapse = ' + '))
        model <- lm(model_formula, data = na.omit(model_data))
        return(model$coefficients)
    })
    output_data <- as.xts(do.call('rbind', fmb_coefs), order.by = main_idx[-1])
    output_coefs <- lapply(output_data, function(coef){ # Regress all coefs on 1
        output_model_data <- cbind(na.omit(coef), 1)
        colnames(output_model_data) <- c('coef_', 'one')
        model <- lm(formula = coef_ ~ one, data = output_model_data) # Explicit to avoid package error
        avg <- model$coefficients[[1]]
        nw <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
        return(c(avg,nw))
    })
    return(output_coefs)
}

# Specify the explained variable data (here one-month ahead returns),
#   and the list of explanatory variables, where the first one in the list is
#   the main independent variable data (the one whose relationship to returns should be
#   investigated). Also provide names for all of the factors in this list.
#   Return a nice data frame, where the main explanatory factor has been ran in combination
#   wih all other factors at one, then one by one, and then independently.
getFMBOutput <- function(monthly_returns_data, explanatory_data, explanatory_data_names){
    if (length(explanatory_data_names) < 2){
        stop('You must provide at least two explanatory factors to get a pretty output.')
    }
    # Static transformation
    main_explanatory_list <- explanatory_data[1] # Main explanatory factor - as a list
    main_explanatory_data <- explanatory_data[[1]] # Main explanatory factor - as xts
    main_explanatory_name <- explanatory_data_names[1]
    regression_coefs <- list()

    # Base FMB regression
    base_fmb <- getFMBRegression(monthly_returns_data, main_explanatory_list, main_explanatory_name)
    regression_coefs[[1]] <- base_fmb

    # FMB regressions with main factor and one other factor, iterate across all
    for (i in 2:length(explanatory_data)){
        exp_factor_data <- explanatory_data[[i]]
        exp_factor_name <- explanatory_data_names[i]
        temp_data <- list(main_explanatory_data, exp_factor_data)
        temp_names <- c(main_explanatory_name, exp_factor_name)
        temp_fmb <- getFMBRegression(monthly_returns_data, temp_data, temp_names)
        regression_coefs[[i]] <- temp_fmb
    }

    # FMB regression with all factors
    all_fmb <- getFMBRegression(monthly_returns_data, explanatory_data, explanatory_data_names)
    all_fmb_idx <- length(explanatory_data) + 1
    regression_coefs[[all_fmb_idx]] <- all_fmb

    # Regression coefficients to an output data frame
    base_dim <- length(explanatory_data_names) + 1
    coefs_m <- matrix(nrow = (2*(base_dim)), ncol = (base_dim)) # Empty matrix
    
    fillCoefs <- function(m, coefs){ # Input an empty matrix and a coefficients to fill it with 
        last_row_idx <- nrow(m)
        # Fill first col
        col_idx <- 1
        temp_coefs <- coefs[[col_idx]]
        m[1,col_idx] <- temp_coefs[[1]][1] # Intercept
        m[1 + 1, col_idx] <- temp_coefs[[1]][2] # N-W intercept t-stat
        m[3, col_idx] <- temp_coefs[[-1]][1] # Vol 
        m[3 + 1, col_idx] <- temp_coefs[[-1]][2] # Vol t-stat

        # Fill middle cols
        for(i in 2:length(explanatory_data)){
            col_idx <- i
            temp_coefs <- unlist(coefs[[col_idx]])
            for(j in 1:4){ # Fill first two, static, factors (intercept, main factor)
                m[j, col_idx] <- temp_coefs[j]
            }
            factor_idx <- 5 + (i - 2) * 2 # Factor intercept row index
            m[factor_idx, col_idx] <- temp_coefs[5] # Factor intercept
            m[factor_idx + 1, col_idx] <- temp_coefs[6] # Factor N-W t-stat

        }

        # Fill last col
        col_idx <- base_dim
        temp_coefs <- unlist(coefs[[col_idx]])
        for(i in 1:last_row_idx){
            m[i, col_idx] <- temp_coefs[i]
        }
        # Round the numbers
        for(col in 1:ncol(m)){
            for(row in 1:nrow(m)){
                m[row,col] <- round(m[row, col], 4)
            }
        }
        return(m)
    }

    # Fill matrix, matrix to data frame, handle row/col names
    coefs_matrix <- fillCoefs(coefs_m, regression_coefs)
    coefs_df <- as.data.frame(coefs_matrix)
    coefs_df[is.na(coefs_df)] <- " "
    metric_col <- c(rep(c('Coefficient', 'N-W t-stat'), base_dim))
    coef_names <- rep(c('Intercept', c(explanatory_data_names)), c(rep(2, base_dim)))
    output_df <- cbind(coef_names, metric_col, coefs_df)
    colnames(output_df) <- c('Factor', 'Metric', 1:(base_dim))

    return(output_df)
}


In [17]:
exp_data <- list(monthly_volatilities, monthly_betas, monthly_sizes,
                monthly_mom_fmb, monthly_illiq_fmb)
exp_data_names <- c('Volatility', 'Beta', 'Size', 'Momentum', 'Illiquidity')
fmb_regression <- getFMBOutput(monthly_returns, exp_data, exp_data_names)

In [18]:
fmb_regression

Factor      Metric      1      2       3       4      5       6      
1  Intercept   Coefficient 0.0031 4e-04   0.011   0.0025 -0.0027  0.0016
2  Intercept   N-W t-stat  0.5975 0.0805  2.4103  0.5258 -0.5964  0.1798
3  Volatility  Coefficient 0.0018 0.0028  0.0016  0.0012 0.0014   0.0015
4  Volatility  N-W t-stat  2.4255 4.2093  1.9874  1.101  2.0287   1.2205
5  Beta        Coefficient        -0.0065                         0.0001
6  Beta        N-W t-stat         -1.2686                         0.0208
7  Size        Coefficient                -0.0033                -0.0024
8  Size        N-W t-stat                 -4.7286                -0.9240
9  Momentum    Coefficient                        0              -0.0049
10 Momentum    N-W t-stat                         0.0029         -0.4845
11 Illiquidity Coefficient                               0.0071   0.0040
12 Illiquidity N-W t-stat                                6.276    0.9605